In [2]:
__author__ = 'Elliot I. Simon'
__email__ = 'ellsim@dtu.dk'
__version__ = 'July 05, 2021'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import glob
import xarray as xr
import scipy as sp
from netCDF4 import Dataset
from netCDF4 import MFDataset

In [4]:
%matplotlib inline
#%matplotlib notebook

In [5]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
plt.rcParams['figure.figsize'] = [10, 8]

In [6]:
# Get list of files
filenames=[]
path = r"R:\Globe\Lidar_data\Onshore_calibrations\sys2\los_measurements\*.nc"
sys = path.split('\\')[-3]
for file in glob.glob(path):
    filenames.append(file)

In [7]:
filenames

['R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_16-24-44_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_16-34-45_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_16-44-45_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_16-54-46_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_17-04-47_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_17-14-48_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_17-24-48_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore_calibrations\\sys2\\los_measurements\\WCS000216_2021-06-11_17-34-49_fixed_387_100m.nc',
 'R:\\Globe\\Lidar_data\\Onshore

In [8]:
def export_data(alldata, rg_list, outpath):

    # Select only interesting channels
    export_df = alldata[['timestamp', 'azimuth', 'elevation', 'cnr', 'radial_wind_speed', 'radial_wind_speed_ci']].to_dataframe()

    # Loop through provided list of range gates
    for rg in rg_list:
        
        # Construct file paths
        outfile = outpath + '\\' + sys + '_' + str(rg) + 'm'+ '.csv'
        outfile_1min = outpath + '\\' + sys + '_' + str(rg) + 'm' + '_1min' + '.csv'
        
        # Select data for specific range gate
        df = export_df.loc[rg, :]
        # Convert string timestamps to datetime64
        df.loc[:, 'timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        # Drop any rows with corrupted data
        df.dropna(how='any', inplace=True)
        # Copy original time column
        df['time'] = df.index
        # Set datetime64 object as timeindex
        df.set_index(df['timestamp'], inplace=True)
        del df['timestamp']
        #print(df.head())
        
        # Create original file and then append further data without repeating headers
        if not os.path.isfile(outfile):
            df.to_csv(outfile, header=True, sep=';', decimal='.')
        else:
            df.to_csv(outfile, mode='a', header=None, sep=';', decimal='.')
        
        # Average to 1Minute and export lower resolution for reduced filesize
        df_1min = df.resample('1Min').mean()
        
        if not os.path.isfile(outfile_1min):
            df_1min.to_csv(outfile_1min, header=True, sep=';', decimal='.')
        else:
            df_1min.to_csv(outfile_1min, mode='a', header=None, sep=';', decimal='.')

In [ ]:
# Process all data
for f in filenames:
    #print(f)
    
    # Get sweep id from metadata
    rootgrp = Dataset(f, "r", format="NETCDF4")
    group = list(rootgrp.groups.keys())[-1]
    # Open netcdf file inside specific sweep group
    alldata = xr.open_dataset(f, group=group, decode_times=False)
    
    # Decide which range gates to extract data
    rg_list = [1000,1500,2000,3000,5000,7000]
    # Make export path
    outpath = path.split('los_measurements')[0] + 'export'
    # Run data export function
    export_data(alldata, rg_list, outpath)

C:\Users\ellsim\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ellsim\AppData\Roaming\Python\Python38\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\ellsim\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i